In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier

df = pd.read_excel("kerevit_mean_final.xlsx")

X = df[['W', 'KB', 'KE', 'AB', 'AE', 'K_Sag', 'K_Sol', 'U_Sag', 'U_Sol', 'KE_Sag', 'KE_Sol']]
y = df['CINSIYET']

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('mlp', MLPClassifier(max_iter = 200))
])

param_grid = {
    'mlp__hidden_layer_sizes': [(32,), (64,), (128,), (32, 32), (64, 64), (128, 128), (32, 32, 32), (64, 64, 64), (128, 128, 128)],
    'mlp__activation': ['relu', 'tanh', 'sigmoid'],
    'mlp__learning_rate_init': [0.001, 0.01, 0.1],
    'mlp__solver': ['adam', 'sgd', 'rmsprop']
}

grid = GridSearchCV(pipe, param_grid, cv = 10, n_jobs = -1, scoring = 'accuracy')
grid.fit(X, y)

print("En iyi parametreler: ", grid.best_params_)
print("En iyi skor: ", grid.best_score_)

C:\Users\Yasin\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Yasin\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


En iyi parametreler:  {'mlp__activation': 'relu', 'mlp__hidden_layer_sizes': (64, 64), 'mlp__learning_rate_init': 0.01, 'mlp__solver': 'sgd'}
En iyi skor:  0.8204545454545455


C:\Users\Yasin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
1350 fits failed out of a total of 2430.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
103 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Yasin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Yasin\anaconda3\lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\Yasin\anaconda3\lib\site-packages\sklearn\pipeline.py", line 476, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "C:

# Eğitim ve Test Aşaması

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import joblib

In [3]:
df = pd.read_excel('kerevit_mean_final.xlsx')

In [4]:
X = df[['W', 'KB', 'KE', 'AB', 'AE', 'K_Sag', 'K_Sol', 'U_Sag', 'U_Sol', 'KE_Sag', 'KE_Sol']]
y = df['CINSIYET']
sira = df['Sira']  # Sıra bilgisini almak

In [5]:
kf = KFold(n_splits = 10, shuffle = False)
conf_matrices = []
fold = 1

In [6]:
results = pd.DataFrame(columns=['Sira', 'Tahmin', 'Gerçek'])

In [7]:
best_params = {
    'activation' : 'relu',
    'hidden_layer_sizes' : (64, 64),
    'learning_rate_init': 0.01,
    'solver': 'sgd',
    'max_iter' : 200
}

In [8]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    sira_test = sira.iloc[test_index]
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    mlp = MLPClassifier(**best_params)
    mlp.fit(X_train, y_train)
    
    model_filename = f'ann_model_fold_{fold}.joblib'
    joblib.dump(mlp, model_filename)
    
    y_pred = mlp.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    conf_matrices.append(cm)
    
    print(f"Fold {fold}:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", cm)
    print("\n")
    
    fold_results = pd.DataFrame({
        'Sira': sira_test,
        'Tahmin': y_pred,
        'Gerçek': y_test
    })
    results = pd.concat([results, fold_results])
    
    fold += 1
    
total_conf_matrix = np.zeros((2, 2), dtype=int) 
for i, cm in enumerate(conf_matrices, 1):
    print(f"Confusion Matrix for Fold {i}:\n{cm}\n")
    total_conf_matrix += cm

print("Total Confusion Matrix:\n", total_conf_matrix)

TN, FP, FN, TP = total_conf_matrix.ravel()

accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP) if (TP + FP) != 0 else 0
recall = TP / (TP + FN) if (TP + FN) != 0 else 0
specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

print(f"Overall Accuracy: {accuracy:.4f}")
print(f"Overall Precision: {precision:.4f}")
print(f"Overall Recall: {recall:.4f}")
print(f"Overall Specificity: {specificity:.4f}")
print(f"Overall F1 Score: {f1_score:.4f}")

results.to_excel('ann_predict.xlsx', index=False)

C:\Users\Yasin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Yasin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Fold 1:
Accuracy: 0.9166666666666666
Classification Report:
               precision    recall  f1-score   support

           D       0.88      1.00      0.93         7
           E       1.00      0.80      0.89         5

    accuracy                           0.92        12
   macro avg       0.94      0.90      0.91        12
weighted avg       0.93      0.92      0.91        12

Confusion Matrix:
 [[7 0]
 [1 4]]


Fold 2:
Accuracy: 0.75
Classification Report:
               precision    recall  f1-score   support

           D       0.75      0.86      0.80         7
           E       0.75      0.60      0.67         5

    accuracy                           0.75        12
   macro avg       0.75      0.73      0.73        12
weighted avg       0.75      0.75      0.74        12

Confusion Matrix:
 [[6 1]
 [2 3]]




C:\Users\Yasin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Yasin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Fold 3:
Accuracy: 0.8181818181818182
Classification Report:
               precision    recall  f1-score   support

           D       0.83      0.83      0.83         6
           E       0.80      0.80      0.80         5

    accuracy                           0.82        11
   macro avg       0.82      0.82      0.82        11
weighted avg       0.82      0.82      0.82        11

Confusion Matrix:
 [[5 1]
 [1 4]]


Fold 4:
Accuracy: 0.9090909090909091
Classification Report:
               precision    recall  f1-score   support

           D       1.00      0.83      0.91         6
           E       0.83      1.00      0.91         5

    accuracy                           0.91        11
   macro avg       0.92      0.92      0.91        11
weighted avg       0.92      0.91      0.91        11

Confusion Matrix:
 [[5 1]
 [0 5]]




C:\Users\Yasin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Yasin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


...

C:\Users\Yasin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Yasin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Fold 7:
Accuracy: 0.7272727272727273
Classification Report:
               precision    recall  f1-score   support

           D       1.00      0.50      0.67         6
           E       0.62      1.00      0.77         5

    accuracy                           0.73        11
   macro avg       0.81      0.75      0.72        11
weighted avg       0.83      0.73      0.71        11

Confusion Matrix:
 [[3 3]
 [0 5]]


Fold 8:
Accuracy: 0.7272727272727273
Classification Report:
               precision    recall  f1-score   support

           D       0.57      1.00      0.73         4
           E       1.00      0.57      0.73         7

    accuracy                           0.73        11
   macro avg       0.79      0.79      0.73        11
weighted avg       0.84      0.73      0.73        11

Confusion Matrix:
 [[4 0]
 [3 4]]




C:\Users\Yasin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Yasin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Fold 9:
Accuracy: 0.7272727272727273
Classification Report:
               precision    recall  f1-score   support

           D       0.83      0.71      0.77         7
           E       0.60      0.75      0.67         4

    accuracy                           0.73        11
   macro avg       0.72      0.73      0.72        11
weighted avg       0.75      0.73      0.73        11

Confusion Matrix:
 [[5 2]
 [1 3]]


Fold 10:
Accuracy: 0.7272727272727273
Classification Report:
               precision    recall  f1-score   support

           D       1.00      0.57      0.73         7
           E       0.57      1.00      0.73         4

    accuracy                           0.73        11
   macro avg       0.79      0.79      0.73        11
weighted avg       0.84      0.73      0.73        11

Confusion Matrix:
 [[4 3]
 [0 4]]


Confusion Matrix for Fold 1:
[[7 0]
 [1 4]]

Confusion Matrix for Fold 2:
[[6 1]
 [2 3]]

Confusion Matrix for Fold 3:
[[5 1]
 [1 4]]

Confusion Matrix

C:\Users\Yasin\AppData\Local\Temp/ipykernel_12396/535685221.py:56: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  results.to_excel('ann_predict.xlsx', index=False)
